# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("cities.csv")
cities_df

,Unnamed: 0,city,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,albany,75,US,1582990460,53,42.60,-73.97,28.99,13.87
1,1,varzelandia,97,BR,1582990479,70,-15.70,-44.03,80.83,6.98
2,2,pisco,75,PE,1582990534,30,-13.70,-76.22,71.60,4.70
3,3,kodiak,90,US,1582990534,79,57.79,-152.41,21.20,5.82
4,4,bathsheba,75,BB,1582990534,61,13.22,-59.52,80.60,14.99
...,...,...,...,...,...,...,...,...,...,...
545,545,verkhnyaya inta,100,RU,1582990613,73,65.98,60.31,22.41,22.50
546,546,naze,75,JP,1582990613,93,28.37,129.48,66.20,5.82
547,547,teguise,20,ES,1582990613,53,29.06,-13.56,73.40,12.75
548,548,masterton,0,NZ,1582990613,84,-40.96,175.66,54.00,2.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_cities_df = cities_df.loc[(cities_df["Max Temp"] < 90) & (cities_df["Max Temp"] > 75) & (cities_df["Humidity"] < 60) 
                                   & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] < 20), :]

narrowed_cities_df = narrowed_cities_df.dropna(how="any")
narrowed_cities_df.reset_index(inplace=True)
del narrowed_cities_df["index"]
del narrowed_cities_df["Unnamed: 0"]
narrowed_cities_df

,city,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,chuy,0,UY,1582990541,42,-33.70,-53.46,84.06,3.13
1,tomatlan,0,MX,1582990551,48,19.93,-105.25,77.05,2.06
2,topchanchi,1,IN,1582990555,37,23.90,86.20,75.38,0.51
3,lebu,0,CL,1582990560,26,-37.62,-73.65,82.20,4.43
4,yenangyaung,0,MM,1582990567,34,20.47,94.88,75.43,5.99
5,jardim,0,BR,1582990570,46,-21.48,-56.14,86.92,4.79
6,lorca,0,ES,1582990570,25,37.67,-1.70,78.01,6.93
7,pijijiapan,0,MX,1582990572,36,15.70,-93.23,82.42,7.70
8,yulara,6,AU,1582990581,17,-25.24,130.99,84.20,9.17
9,nakhon phanom,0,TH,1582990581,59,17.41,104.78,77.00,2.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = []


for city in range(len(narrowed_cities_df)):
    lat = narrowed_cities_df.loc[city]["Latitude"]
    lng = narrowed_cities_df.loc[city]["Longitude"]
    
    params = {
        "location": f"{lat}, {lng}",
        "radius": 5000, 
        "types": "hotel",
        "key": g_key,
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params)
    requested_json = requested.json()
#     print(json.dumps(requested_json, indent=4, sort_keys=True))
    try: 
        hotel_df.append(requested_json["results"][0]["name"])
    except:
        hotel_df.append("")
        

narrowed_cities_df["Hotel Name"] = hotel_df
narrowed_cities_df = narrowed_cities_df.dropna(how="any")
narrowed_cities_df.head()

,city,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,chuy,0,UY,1582990541,42,-33.70,-53.46,84.06,3.13,Chuy
1,tomatlan,0,MX,1582990551,48,19.93,-105.25,77.05,2.06,Tomatlán
2,topchanchi,1,IN,1582990555,37,23.90,86.20,75.38,0.51,Topchanchi
3,lebu,0,CL,1582990560,26,-37.62,-73.65,82.20,4.43,Lebu
4,yenangyaung,0,MM,1582990567,34,20.47,94.88,75.43,5.99,Yenangyaung


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_cities_df.iterrows()]
locations = narrowed_cities_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display Map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))